In [40]:
from cb25d.batch import run_batch_simulation
from cb25d.interactive import run_interactive_simulation
from cb25d.notebook import init
from cb25d.simulation_impl_original import (
    SimulationImplOriginal,
    SimulationRecorderOriginal,
    SimulationRendererOriginal,
    generate_initial_conditions,
)

init()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
run_interactive_simulation(
    SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=0.6,
        c_gamma_att=0.6,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=0,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    SimulationRendererOriginal(
        pos_size=2 / 4,
        pos_color=(0, 255, 0),
        dir_len=5 / 4,
        dir_width=1 / 4,
        dir_color=(50, 255, 50),
    ),
    scale=2**-5,
)

In [37]:
run_interactive_simulation(
    SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=0.2,
        c_gamma_att=0.37,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=1,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    SimulationRendererOriginal(
        pos_size=2 / 8,
        pos_color=(0, 255, 0),
        dir_len=5 / 8,
        dir_width=1 / 8,
        dir_color=(50, 255, 50),
    ),
    scale=2**-5,
)


In [ ]:
run_batch_simulation(
    SimulationImplOriginal(
        c_eta=0.8,
        c_gamma_ali=0.2,
        c_gamma_att=0.37,
        c_gamma_rand=0.2,
        c_k=1,
        c_l_ali=3,
        c_tau_0=0.8,
        **generate_initial_conditions(
            seed=1,
            n=100,
            l_att=3,
            tau_n_mean=1,
            tau_n_std=1,
        ),
    ),
    rec := SimulationRecorderOriginal(skip_first_n=1000),
    steps=2000,
)
print(rec.dispersion)
print(rec.polarization)

8.019416335863994
0.5992334058296201
